In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
t = np.arange(0, 2500)
x = np.sin(0.02 * t) + np.random.rand(2500) * 2

In [3]:
train, test = x[0:2000], x[:2000]

In [4]:
def convertToDataset(data, step):
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

In [5]:
train_step = 10
test_step = 20

x_train, y_train = convertToDataset(train, train_step)
x_test, y_test = convertToDataset(test, test_step)

In [6]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [7]:
x_train.shape

(1990, 10, 1)

In [8]:
x_test.shape

(1980, 20, 1)

In [9]:
def Build_Model(sequence_length):
  model = Sequential()
  model.add(SimpleRNN(units=64, input_shape=(sequence_length, 1), activation = "tanh"))
  model.add(Dense(1))
  return model

In [10]:
model = Build_Model(10)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 64)                4224      
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 4289 (16.75 KB)
Trainable params: 4289 (16.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.input

<KerasTensor: shape=(None, 10, 1) dtype=float32 (created by layer 'simple_rnn_input')>

In [12]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [13]:
checkpoint_dir = './notebook02_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = ModelCheckpoint(filepath = checkpoint_prefix, save_weights_only=True)

In [14]:
history = model.fit(x_train, y_train, epochs=100, batch_size=16, verbose=2, callbacks=[checkpoint_callback])

Epoch 1/100
125/125 - 1s - loss: 0.4729 - 1s/epoch - 8ms/step
Epoch 2/100
125/125 - 0s - loss: 0.4056 - 229ms/epoch - 2ms/step
Epoch 3/100
125/125 - 0s - loss: 0.3966 - 232ms/epoch - 2ms/step
Epoch 4/100
125/125 - 0s - loss: 0.3956 - 241ms/epoch - 2ms/step
Epoch 5/100
125/125 - 0s - loss: 0.3888 - 227ms/epoch - 2ms/step
Epoch 6/100
125/125 - 0s - loss: 0.3859 - 220ms/epoch - 2ms/step
Epoch 7/100
125/125 - 0s - loss: 0.3898 - 233ms/epoch - 2ms/step
Epoch 8/100
125/125 - 0s - loss: 0.3833 - 221ms/epoch - 2ms/step
Epoch 9/100
125/125 - 0s - loss: 0.3845 - 222ms/epoch - 2ms/step
Epoch 10/100
125/125 - 0s - loss: 0.3785 - 230ms/epoch - 2ms/step
Epoch 11/100
125/125 - 0s - loss: 0.3799 - 317ms/epoch - 3ms/step
Epoch 12/100
125/125 - 0s - loss: 0.3825 - 334ms/epoch - 3ms/step
Epoch 13/100
125/125 - 0s - loss: 0.3781 - 386ms/epoch - 3ms/step
Epoch 14/100
125/125 - 0s - loss: 0.3853 - 324ms/epoch - 3ms/step
Epoch 15/100
125/125 - 0s - loss: 0.3749 - 369ms/epoch - 3ms/step
Epoch 16/100
125/125 -

In [15]:
tf.train.latest_checkpoint(checkpoint_dir)

'./notebook02_training_checkpoints/ckpt_100'

In [16]:
model = Build_Model(20)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.compile(loss='mean_squared_error', optimizer='adam')

In [17]:
train_score = model.evaluate(x_test, y_test, verbose=0)
print(train_score)

0.532565712928772
